# Vector Database per il Testo con LanceDB

Questo notebook dimostra come utilizzare LanceDB come vector database per dati testuali. Vedremo come:

1. Installare le dipendenze necessarie
2. Caricare e preparare un dataset testuale
3. Suddividere il testo in chunk
4. Generare embedding per i chunk di testo
5. Caricare gli embedding in LanceDB
6. Eseguire query di ricerca semantica

## Cos'è un Vector Database?

Un vector database è un tipo di database ottimizzato per memorizzare e cercare vettori di embedding. Gli embedding sono rappresentazioni numeriche di dati (come testo o immagini) che catturano il significato semantico. I vector database consentono di eseguire ricerche semantiche efficienti, trovando elementi simili in base alla loro vicinanza nello spazio vettoriale.

## 1. Installazione delle dipendenze

Per prima cosa, installiamo le librerie necessarie:

In [1]:
!pip install lancedb sentence-transformers datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## 2. Importazione delle librerie

In [ ]:
import os
import lancedb
import pandas as pd
import numpy as np
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

## 3. Caricamento di un dataset testuale

Utilizzeremo un dataset di articoli di Wikipedia in italiano dalla libreria Hugging Face datasets.

In [ ]:
# Carichiamo un dataset di esempio (articoli di Wikipedia in italiano)
dataset = load_dataset("wikimedia/wikipedia", "20231101.it", split="train", streaming=True)
# Prendiamo solo i primi 50 articoli per questo esempio
articles = list(dataset.take(50))

# Visualizziamo la struttura di un articolo
print("Struttura di un articolo:")
for key in articles[0].keys():
    print(f"- {key}")

# Visualizziamo un esempio di titolo e l'inizio del testo
print(f"\nTitolo: {articles[0]['title']}")
print(f"Inizio del testo: {articles[0]['text'][:300]}...")

## 4. Chunking del testo

Per gestire testi lunghi, è necessario suddividerli in chunk più piccoli. Questo processo è chiamato "chunking" ed è fondamentale per l'elaborazione efficiente del testo nei vector database.

In [ ]:
def chunk_text(text, chunk_size=200, overlap=50):
    """
    Suddivide un testo in chunk di dimensione specificata con sovrapposizione.

    Args:
        text (str): Il testo da suddividere
        chunk_size (int): Numero di parole per chunk
        overlap (int): Numero di parole di sovrapposizione tra chunk consecutivi

    Returns:
        list: Lista di chunk di testo
    """
    words = text.split()
    chunks = []

    if len(words) <= chunk_size:
        return [text]

    i = 0
    while i < len(words):
        # Prendiamo chunk_size parole o fino alla fine del testo
        chunk_words = words[i:i + chunk_size]
        chunk = " ".join(chunk_words)
        chunks.append(chunk)

        # Avanziamo di (chunk_size - overlap) parole
        i += (chunk_size - overlap)

    return chunks

In [ ]:
# Prepariamo i dati: creiamo chunk per ogni articolo
all_chunks = []
chunk_metadata = []

for i, article in enumerate(tqdm(articles, desc="Chunking articles")):
    title = article['title']
    text = article['text']

    # Suddividiamo il testo in chunk
    chunks = chunk_text(text)

    # Aggiungiamo ogni chunk alla lista con i relativi metadati
    for j, chunk in enumerate(chunks):
        all_chunks.append(chunk)
        chunk_metadata.append({
            'article_id': i,
            'title': title,
            'chunk_id': j,
            'total_chunks': len(chunks)
        })

print(f"Totale articoli: {len(articles)}")
print(f"Totale chunks: {len(all_chunks)}")
print(f"\nEsempio di chunk: {all_chunks[10][:150]}...")

## 5. Generazione degli embedding

Utilizzeremo un modello di SentenceTransformers pre-addestrato per generare gli embedding dei nostri chunk di testo. Per il testo in italiano, utilizziamo un modello multilingue.

In [ ]:
# Carichiamo un modello di embedding multilingue
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Generiamo gli embedding per tutti i chunk
embeddings = []

# Processiamo i chunk in batch per efficienza
batch_size = 32
for i in tqdm(range(0, len(all_chunks), batch_size), desc="Generating embeddings"):
    batch = all_chunks[i:i+batch_size]
    batch_embeddings = model.encode(batch)
    embeddings.extend(batch_embeddings)

print(f"Dimensione di un embedding: {len(embeddings[0])}")

## 6. Creazione del Vector Database con LanceDB

Ora creiamo un database LanceDB e carichiamo i nostri embedding.

In [ ]:
# Creiamo un dataframe con i chunk, i metadati e gli embedding
data = []
for i in range(len(all_chunks)):
    data.append({
        'text': all_chunks[i],
        'article_id': chunk_metadata[i]['article_id'],
        'title': chunk_metadata[i]['title'],
        'chunk_id': chunk_metadata[i]['chunk_id'],
        'total_chunks': chunk_metadata[i]['total_chunks'],
        'vector': embeddings[i]
    })

df = pd.DataFrame(data)
print(f"Dataframe shape: {df.shape}")
df.head(2)

In [ ]:
# Creiamo un database LanceDB
db_path = "./lancedb_wiki"
db = lancedb.connect(db_path)

# Creiamo una tabella per i nostri dati
table_name = "wiki_articles"

# Se la tabella esiste già, la eliminiamo
if table_name in db.table_names():
    db.drop_table(table_name)

# Creiamo la tabella con i nostri dati
table = db.create_table(table_name, data=df, mode="overwrite")

print(f"Tabella '{table_name}' creata con successo!")

## 7. Esecuzione di query semantiche

Ora possiamo eseguire query semantiche sul nostro vector database.

In [ ]:
def semantic_search(query_text, top_k=5):
    """
    Esegue una ricerca semantica nel vector database.

    Args:
        query_text (str): Il testo della query
        top_k (int): Numero di risultati da restituire

    Returns:
        list: Lista dei risultati più rilevanti
    """
    # Generiamo l'embedding per la query
    query_embedding = model.encode(query_text)

    # Eseguiamo la ricerca vettoriale
    results = table.search(query_embedding).limit(top_k).to_pandas()

    return results

In [ ]:
# Esempio di query semantica
query = "storia dell'Italia antica"
results = semantic_search(query)

print(f"Query: '{query}'\n")
print("Risultati più rilevanti:")
for i, row in results.iterrows():
    print(f"\n--- Risultato {i+1} ---")
    print(f"Titolo: {row['title']}")
    print(f"Chunk: {row['chunk_id']+1}/{row['total_chunks']}")
    print(f"Testo: {row['text'][:200]}...")
    print(f"Distanza: {row['_distance']:.4f}")

In [ ]:
# Proviamo un'altra query
query = "scienza e tecnologia moderna"
results = semantic_search(query)

print(f"Query: '{query}'\n")
print("Risultati più rilevanti:")
for i, row in results.iterrows():
    print(f"\n--- Risultato {i+1} ---")
    print(f"Titolo: {row['title']}")
    print(f"Chunk: {row['chunk_id']+1}/{row['total_chunks']}")
    print(f"Testo: {row['text'][:200]}...")
    print(f"Distanza: {row['_distance']:.4f}")

## 8. Filtraggio dei risultati

LanceDB supporta anche il filtraggio dei risultati in base ai metadati.

In [ ]:
def filtered_search(query_text, title_filter=None, top_k=5):
    """
    Esegue una ricerca semantica con filtro sul titolo.

    Args:
        query_text (str): Il testo della query
        title_filter (str): Filtro sul titolo (opzionale)
        top_k (int): Numero di risultati da restituire

    Returns:
        list: Lista dei risultati più rilevanti
    """
    # Generiamo l'embedding per la query
    query_embedding = model.encode(query_text)

    # Prepariamo la query
    search_query = table.search(query_embedding)

    # Applichiamo il filtro se specificato
    if title_filter:
        search_query = search_query.where(f"title LIKE '%{title_filter}%'")

    # Eseguiamo la ricerca
    results = search_query.limit(top_k).to_pandas()

    return results

In [ ]:
# Esempio di ricerca con filtro
query = "eventi importanti"
title_filter = "storia"
results = filtered_search(query, title_filter)

print(f"Query: '{query}' (filtro titolo: '{title_filter}')\n")
print("Risultati più rilevanti:")
for i, row in results.iterrows():
    print(f"\n--- Risultato {i+1} ---")
    print(f"Titolo: {row['title']}")
    print(f"Chunk: {row['chunk_id']+1}/{row['total_chunks']}")
    print(f"Testo: {row['text'][:200]}...")
    print(f"Distanza: {row['_distance']:.4f}")

## 9. Aggiornamento del Vector Database

LanceDB supporta anche l'aggiornamento incrementale dei dati.

In [ ]:
# Creiamo un nuovo chunk da aggiungere
new_text = "L'intelligenza artificiale è un campo dell'informatica che si occupa di creare sistemi in grado di svolgere compiti che normalmente richiederebbero l'intelligenza umana. Questi compiti includono il riconoscimento vocale, il processo decisionale, la traduzione tra lingue e la percezione visiva."
new_title = "Intelligenza Artificiale"

# Generiamo l'embedding
new_embedding = model.encode(new_text)

# Creiamo un dataframe con il nuovo dato
new_data = pd.DataFrame([
    {
        'text': new_text,
        'article_id': len(articles),  # Nuovo ID
        'title': new_title,
        'chunk_id': 0,
        'total_chunks': 1,
        'vector': new_embedding
    }
])

# Aggiungiamo il nuovo dato alla tabella
table.add(new_data)

print(f"Nuovo chunk aggiunto al database!")

In [ ]:
# Verifichiamo che il nuovo dato sia stato aggiunto
query = "intelligenza artificiale e machine learning"
results = semantic_search(query)

print(f"Query: '{query}'\n")
print("Risultati più rilevanti:")
for i, row in results.iterrows():
    print(f"\n--- Risultato {i+1} ---")
    print(f"Titolo: {row['title']}")
    print(f"Chunk: {row['chunk_id']+1}/{row['total_chunks']}")
    print(f"Testo: {row['text'][:200]}...")
    print(f"Distanza: {row['_distance']:.4f}")

## 10. Conclusioni

In questo notebook abbiamo visto come:

1. Caricare e preparare un dataset testuale
2. Suddividere il testo in chunk
3. Generare embedding per i chunk di testo
4. Creare un vector database con LanceDB
5. Eseguire query semantiche
6. Filtrare i risultati in base ai metadati
7. Aggiornare il database con nuovi dati

I vector database come LanceDB sono strumenti potenti per la ricerca semantica e possono essere utilizzati in molte applicazioni, come motori di ricerca, sistemi di raccomandazione, chatbot e molto altro.

## Esercizi aggiuntivi

1. Prova a utilizzare un modello di embedding diverso (ad esempio, un modello specifico per l'italiano)
2. Sperimenta con diverse strategie di chunking (dimensione dei chunk, sovrapposizione)
3. Implementa una funzione per recuperare l'intero articolo dato un chunk rilevante
4. Crea un'interfaccia utente semplice per la ricerca semantica